## Obtaining baseline results using Yolo models 

### Data

#### Converting Geojson to YOLO

In [10]:
import os
import numpy as np
from glob import glob
from PIL import Image

def fix_path(path: str) -> str:
    return path.replace("\\", "/")

# Directories
labels_dir = fix_path(r"O:\Capstone_2\Marine-Debris-Detection\NASA_Planet_Data\labels")
source_dir = fix_path(r"O:\Capstone_2\Marine-Debris-Detection\NASA_Planet_Data\source")
output_dir = fix_path("O:/Capstone_2/Marine-Debris-Detection/yolo_labels")

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)


In [17]:

# Class mapping
class_map = {
    "marine_debris": 0  # Update with additional classes if needed
}

# Normalize bounding box coordinates
def normalize_bbox(bbox, img_width, img_height):
    x_min, y_min, x_max, y_max = bbox
    x_center = (x_min + x_max) / 2 / img_width
    y_center = (y_min + y_max) / 2 / img_height
    width = (x_max - x_min) / img_width
    height = (y_max - y_min) / img_height
    return x_center, y_center, width, height

# Process .npy files
npy_files = glob(os.path.join(labels_dir, "*.npy"))

for npy_file in npy_files:
    # Load .npy data
    data = np.load(npy_file, allow_pickle=True).tolist()

    # Get corresponding image dimensions
    base_name = os.path.splitext(os.path.basename(npy_file))[0]
    image_path = os.path.join(source_dir, base_name + ".jpg")
    
    if not os.path.exists(image_path):
        print(f"Image not found for {base_name}, skipping...")
        continue

    with Image.open(image_path) as img:
        img_width, img_height = img.size

    # Create YOLO label file
    yolo_file_path = os.path.join(output_dir, base_name + ".txt")
    with open(yolo_file_path, "w") as yolo_file:
        for entry in data:
            x_min, y_min, x_max, y_max, class_id = entry
            if class_id > 0:
                class_id -= 1

            # Normalize the bounding box coordinates
            x_center, y_center, width, height = normalize_bbox(
                (x_min, y_min, x_max, y_max), img_width, img_height
            )

            # Write to YOLO file
            yolo_file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

print("Conversion complete!")


Conversion complete!
